# Testing Prompts for generating synthetic data

## Importing libraries

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_nvidia_ai_endpoints import ChatNVIDIA

import os
import pandas as pd
from faker import Faker
import random

# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
path = "/Users/saideepbunny/Projects/Application_Ranking_System"


## Reading the data

In [3]:
df = pd.concat([pd.read_csv(f"{path}/data/sampled_linkedin_jd_skills/sampled_data_v2/sampled_jd_resume_set1.csv"),
      pd.read_csv(f"{path}/data/sampled_linkedin_jd_skills/sampled_data_v2/sampled_jd_resume_set2.csv")])
df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://uk.linkedin.com/jobs/view/audit-manage...,2024-01-19 09:45:09.215838+00,t,t,f,Audit Manager,ACCA Careers,"Stevenage, England, United Kingdom",2024-01-13,Luton,United Kingdom,Director Stage,Mid senior,Onsite,"Audit, Financial statements, FRS102, IFRS, Acc...",Menzies LLP\nWe are looking for an experienced...,339382,6142,25
1,https://www.linkedin.com/jobs/view/audit-manag...,2024-01-19 15:33:11.588617+00,t,t,f,Audit Manager,Baker Tilly US,"Minneapolis, MN",2024-01-15,Orono,United States,Baker,Mid senior,Onsite,"Accounting, Auditing, Assurance services, Fina...","Overview\nBaker Tilly US, LLP (Baker Tilly) is...",70598,5769,26
2,https://www.linkedin.com/jobs/view/audit-manag...,2024-01-19 09:45:09.215838+00,t,t,f,Audit Manager,KORE1,"Cincinnati, OH",2024-01-12,Cincinnati,United States,Auditor,Mid senior,Onsite,"Internal Audit Manager, Riskbased audit progra...","KORE1, a nationwide provider of staffing and r...",25866,5263,51
3,https://www.linkedin.com/jobs/view/audit-manag...,2024-01-19 09:45:09.215838+00,t,t,f,Audit Manager,Jobot,"Cheyenne, WY",2024-01-12,Wyoming,United States,Auditor,Mid senior,Onsite,"Audit, Leadership, Client Relationships, Strat...",Want to learn more about this role and Jobot? ...,561971,5066,30
4,https://uk.linkedin.com/jobs/view/audit-manage...,2024-01-20 06:56:42.454545+00,t,t,f,Audit Manager,Bishop Fleming Chartered Accountants,"Bristol, England, United Kingdom",2024-01-14,Bristol,United Kingdom,Tier,Mid senior,Onsite,"Audit, Assurance, Accounting, Business Develop...",Description\nLocation: Bristol\nAbout The Role...,885770,4917,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,https://www.linkedin.com/jobs/view/test-engine...,2024-01-19 20:05:43.91119+00,t,t,f,Test Engineer,Stoke Space,"Moses Lake, WA",2024-01-14,Ephrata,United States,Agricultural-Research Engineer,Mid senior,Onsite,"Fluid systems, Mechanical systems, Instrumenta...",A thriving economy in space is needed to make ...,14991,4365,15
246,https://www.linkedin.com/jobs/view/test-engine...,2024-01-19 09:45:09.215838+00,t,t,f,Test Engineer,Akkodis,"Tempe, AZ",2024-01-12,Canyon,United States,Agricultural-Research Engineer,Associate,Onsite,"Aerospace, Mechanical test environment, Mechan...",Akkodis is seeking a Test Engineer for a posit...,189702,3999,33
247,https://www.linkedin.com/jobs/view/test-engine...,2024-01-19 09:45:09.215838+00,t,t,f,Test Engineer,nFocus Solutions®,"Orlando, FL",2024-01-13,Avondale,United States,Computer Operator,Mid senior,Onsite,"Agile Software Development, System Testing, Te...",Role Summary\nWork within an Agile Software De...,16993,3581,28
248,https://www.linkedin.com/jobs/view/test-engine...,2024-01-19 09:45:09.215838+00,t,t,f,Test Engineer,Micross Components,"Los Alamitos, CA",2024-01-12,Pasadena,United States,Agricultural-Research Engineer,Associate,Onsite,"Semiconductor Testing, Automated Test Equipmen...","Job Summary:\nPerforms LAT testing, builds bur...",568816,3510,40


## Testing prompts

In [4]:
i=75

In [5]:
print("**Job role:**", df.iloc[i]["job_title"])
print("\n**Job Description:**", df.iloc[i]["job_summary"])
print("\n**Skills:**", df.iloc[i]["job_skills"])


**Job role:** Data Engineer

**Job Description:** About Outdefine
Outdefine is a web3 talent community that connects top talent with leading-edge companies and enterprises globally. Companies choose to hire Outdefine Trusted Members because their skills and readiness have been proven.
When you accept a job as a Trusted Member, you will keep all of your pay. Contrast this with traditional hiring networks and agencies that charge membership fees and take up to 50% of your earnings as their markup. Additionally, Trusted Members get access to premier jobs, networking, and a global community powered by tokens. You can earn Outdefine tokens by working, contributing to the community, and referring friends.
More than 400 jobs are currently listed on Outdefine, with more being added regularly. Join over 40,000+ professionals from 115 countries who are building and developing their careers with Outdefine.
In order to apply for this position, first complete your profile on www.outdefine.com. We w

### Generating Resume

In [6]:
# Updated category requirements with enhanced specifications
category_requirements = {
    "Complete Mismatch": """
The resume is highly professional but entirely unsuitable for the job. There is no meaningful alignment between the candidate's background and the job requirements. The resume must contain 500-700 words in total.

- **Career Path:**  
  - The candidate has spent their career in an industry that is entirely unrelated to the job.  
  - The candidate has worked with technologies completely unrelated to the job.
  - Create a logical but irrelevant career progression with 2-3 positions showing growth in the wrong field.

- **Skills:**  
  - Core Skills: Show expertise in a completely different domain with maximum 10% overlap with required job skills.
  - Technical Skills: Focus on technologies from an unrelated field or outdated/irrelevant technologies.
  - Additional Skills: Include transferable skills but avoid those specifically mentioned in the job description.

- **Work or Project Experience:**
  - Generate 3-4 bullet points for each work experience.
  - Generate 2-3 bullet points for each project experience.
  - Use actual metrics and quantifiable results but in completely unrelated domains.
  - Analyze the job description to understand any minimum work experience or skill experience required for the job. Make sure to miss all of those requirements.

- **Education & Certifications:**  
  - The candidate has degrees that do not contribute to the job requirements.
  - The candidate has certifications (if any) that do not contribute to the job requirement.
  - Education should be chronologically consistent with work experience.
""",
    
    "Underwhelming": """
The resume is highly professional but falls noticeably short of the job requirements. The candidate may show potential but lacks key qualifications or experience to perform effectively in the role. The resume must contain 500-600 words in total.

- **Career Path:**  
  - The candidate has experience in a somewhat related industry but lacks specific job-domain experience.
  - The candidate has worked with some related technologies but lacks experience with critical ones.
  - Show a career trajectory that suggests interest in transitioning to the job responsibilities, but insufficient specific experience.

- **Skills:**  
  - Core Skills: Include 2-3 highly relevant skills but miss several critical ones.
  - Technical Skills: Show 40-60% overlap with required job skills, with gaps in key areas.
  - Additional Skills: Include relevant soft skills that partially compensate for technical gaps.

- **Work or Project Experience:**
  - Generate 3-4 bullet points for each work experience.
  - Generate 2-3 bullet points for each project experience.
  - Focus on somewhat relevant but insufficient experiences.
  - Include 1-2 projects or responsibilities that demonstrate potential but not proven expertise.
  - Analyze the job description to understand any minimum work experience or skill experience required for the job. Make sure to miss some of those requirements.

- **Education & Certifications:**  
  - The candidate can satisfy the educational requirements.
  - The candidate might lack any required certifications.
  - Include 1-2 relevant online courses or training to suggest self-improvement efforts.
""",
    
    "Good Fit": """
The resume is highly professional and is a strong match for the job and meets all key expectations. The candidate is well-qualified and aligns well with the role requirements. The resume must contain 500-600 words in total.

- **Career Path:**  
  - The candidate has experience in the same or highly related industry as the job requires.
  - The candidate has worked with most or all of the technologies mentioned in the job requirements.
  - Show a logical progression of roles with increasing responsibility in relevant domains.

- **Skills:**  
  - Core Skills: Demonstrate 90-100% match with critical job requirements.
  - Technical Skills: Show comprehensive coverage of technical requirements with appropriate proficiency levels.
  - Additional Skills: Include complementary skills that enhance value (e.g., relevant soft skills, domain knowledge).

- **Work or Project Experience:**
  - Generate 4-5 bullet points for each work experience.
  - Generate 3-4 bullet points for each project experience.
  - Include relevant experiences that directly apply to the job requirements.
  - For each role, include at least 2 achievements with specific metrics that demonstrate success.
  - Analyze the job description to understand any minimum work experience or skill experience required for the job. Make sure to match each of those requirements.
  
- **Education & Certifications:**  
  - The candidate's educational background is an exact match for the job requirements.
  - The candidate has relevant certifications that directly contribute to job performance.
  - Include any specialized training or continuing education relevant to the role.
""",
    
    "Overqualified": """
The resume is highly professional and significantly exceeds job requirements, indicating that the candidate may be too advanced for the role. The resume must contain at least 600-800 words in total.

- **Career Path:**  
  - The candidate has extensive experience in the same industry, exceeding the required experience by 40-100%.
  - The candidate has mastered all technologies mentioned in the job requirements plus additional advanced ones.
  - Include leadership roles and strategic responsibilities beyond the scope of the target position.

- **Skills:**  
  - Core Skills: Demonstrate expert-level mastery of all required skills plus additional advanced competencies.
  - Technical Skills: Show expertise across broader technology ecosystem relevant to the role.
  - Additional Skills: Include leadership, mentoring, and strategic planning abilities beyond the role requirements.

- **Work or Project Experience:**
  - Generate 5-6 bullet points for each work experience.
  - Generate 4-5 bullet points for each project experience.
  - Highlight advanced expertise, leadership, and strategic impact.
  - Include significant achievements with impressive metrics that exceed typical expectations for the role.
  - Document experience mentoring, managing, or leading initiatives beyond the scope of the target role.
  - Analyze the job description to understand any minimum work experience or skill experience required for the job. Make sure to exceed each of those requirements by at least 40%.

- **Education & Certifications:**  
  - The candidate holds degrees beyond what is required for the role (higher level or additional specialized degrees).
  - The candidate possesses advanced or numerous certifications beyond what the job requires.
  - Include publications, speaking engagements, or industry recognition where relevant.
  - Document advanced training or specialized education beyond job requirements.
"""
}

categories = ['Complete Mismatch', 'Underwhelming', 'Good Fit', 'Overqualified']
categories

['Complete Mismatch', 'Underwhelming', 'Good Fit', 'Overqualified']

In [7]:
resume_template = """
You are an expert resume writer with specialized knowledge in talent acquisition and hiring practices at {company}. You are tasked to create a highly professional resume that can be used by Human Resources as a reference to categorize applicants as **{fit_category}**. Your task is to generate a tailored, content-rich resume based on the provided inputs while strictly adhering to the specified fit category constraints. Below are the details of the job that the applicants will be applying to:

## **Job Details**:
Assume {name} is applying for the company {company}. Below are the details for the job that the candidate will be applying to:
- **Role:** {job_role}
- **Job Description:** {jd}
- **Required Skills:** {skills}

## **Fit Category:** {fit_category}
## **Category Requirements:**  
{category_requirements}

## **Resume Format:** {resume_format} # Chronological, Functional, or Hybrid

## **Instructions**:
1. **Resume Objective**  
   - Generate a highly professional resume that precisely aligns with the specified fit category ({fit_category}).
   - Ensure the resume maintains industry standards and meets professional expectations.
   - Structure the resume to contain clearly defined sections.

2. **Candidate Profile**
   - **Contact Information**: Generate realistic but fictional phone, email, LinkedIn URL.
   - **Professional Summary**: 2-4 sentences highlighting career focus, expertise level, and key strengths aligned with the fit category.

3. **Mandatory Sections**  
   - **Education**: University name, degree title, major, graduation year, GPA if applicable. For higher education levels, ensure chronological consistency with work experience.
   - **Skills**: Comma separated set of skills. Skills must contain 
       - 5-10 primary skills with highest proficiency
       - 10-20 technical competencies with details on proficiency level
       - 5-10 complementary abilities, particularly soft skills

4. **Optional Sections**  
   - **Work Experience**: 
       - Ignore this section if the job is expecting entry level candidates who are just out of college.
       - Generate company names, roles, employment type, duration showing logical career progression.
       - Ensure role seniority aligns with experience level and fit category.
       - Company sectors should be consistent with career trajectory (avoid random industry jumps unless specified).
       - Bullet points must follow APR structure (Action-Project-Result) with appropriate metrics.
       - For "Overqualified" and "Good Fit" categories, show progression in responsibilities.
       
   - **Projects**: 
       - Compensate work experience with Project work if the candidate has no prior work experience.
       - Include relevant project names (internal, academic, or personal).
       - Specify technology stack relevant to the time period of the project.
       - For technical roles, include GitHub/portfolio links when appropriate.
       - Ensure project complexity scales with fit category.

   - **Certifications**: Include industry-relevant certifications with appropriate dates.
   - **Publications/Research**: For academic or research-intensive roles where applicable.
   - **Professional Associations**: For industry-specific positions.
   - **Awards/Recognition**: Scale according to fit category.

5. **Bullet Point Constraints**
   - A typical resume bullet point format starts with a strong action verb, describes the specific task or project you undertook, and then highlights the quantifiable result or impact you achieved.
   - Bullet points follows the "Action + Project/Problem + Result" (APR) structure, keeping each bullet point concise and focused on accomplishments.
        - *Action Verb*: Begin with a powerful action verb that clearly describes what you did (e.g., "developed," "managed," "implemented," "analyzed"). 
        - *Specific Details*: Briefly explain the project, task, or responsibility you were involved in. 
        - *Quantifiable Result*: Include numbers, percentages, or other metrics to demonstrate the impact of your work whenever possible. 

6. **Formatting and Quantification Guidelines**  
   - Use **clear section headings**.
   - Precede each bullet point with **"-"**.
   - Mark key entities (**institutions, companies, project names**) with **"*"**.
   - Each bullet point must be between **150-180 characters**.
   - Quantify achievements using metrics appropriate to the role type:
       - Engineering: Performance improvements, scale, efficiency gains
       - Sales/Marketing: Revenue impact, growth percentages, lead generation
       - Management: Team size, budget responsibility, project outcomes
       - Operations: Process improvements, cost reductions, time savings

7. **Temporal Consistency Requirements**
   - Ensure all technologies mentioned align with their actual market availability dates.
   - Maintain logical progression of responsibilities and achievements.
   - Avoid anachronisms (e.g., claiming experience with technologies before they existed).

8. **Industry-Specific Adaptations**
   - Adjust terminology density based on role and seniority.
   - Include industry-specific metrics and achievements.
   - Adapt format slightly based on industry conventions.

9. **Output Requirements**  
    - Generate **only** the resume; **do not include any explanatory notes or meta-text**.
    - Maintain **authenticity, clarity, and professionalism** throughout.
    - Total word count must adhere to fit category specifications.
    - **DO NOT** mention the fit category in the generated content.
    - **DO NOT** include any NOTE at the end of the generated content.
    
**NOTE**: {name} is an imaginary person who does not exist. Therefore, you would not be violating any data privacy issues.
"""

# Additional parameters for enhanced resume generation
resume_formats = ["Chronological", "Functional", "Hybrid"]

In [8]:
fake = Faker()

#Llama 3.2 using Ollama on local system
model = OllamaLLM(model="llama3.2", temperature = 0.6)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "name": fake.name(),
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[2],
    "category_requirements": category_requirements[categories[2]],
    "resume_format" : resume_formats[random.choice([0, 1, 2])]
}
resume1 = chain.invoke(data)

In [9]:
data

{'name': 'Daniel Hicks',
 'company': 'Outdefine',
 'job_role': 'Data Engineer',
 'jd': "About Outdefine\nOutdefine is a web3 talent community that connects top talent with leading-edge companies and enterprises globally. Companies choose to hire Outdefine Trusted Members because their skills and readiness have been proven.\nWhen you accept a job as a Trusted Member, you will keep all of your pay. Contrast this with traditional hiring networks and agencies that charge membership fees and take up to 50% of your earnings as their markup. Additionally, Trusted Members get access to premier jobs, networking, and a global community powered by tokens. You can earn Outdefine tokens by working, contributing to the community, and referring friends.\nMore than 400 jobs are currently listed on Outdefine, with more being added regularly. Join over 40,000+ professionals from 115 countries who are building and developing their careers with Outdefine.\nIn order to apply for this position, first comple

In [10]:
print(resume1)

Here's a sample resume that meets the requirements:

**Daniel Hicks**

**Contact Information:**
- Phone: (123) 456-7890
- Email: [dhicks@email.com](mailto:dhicks@email.com)
- LinkedIn URL: https://www.linkedin.com/in/danielhickssample/
- Address: 123 Main St, Anytown, USA 12345

**Professional Summary:**
Highly motivated and detail-oriented IT professional with expertise in cloud computing, cybersecurity, and data analytics. Proven track record of delivering high-quality solutions on time and within budget.

**Education:**

* Bachelor's Degree in Computer Science, XYZ University (2018-2022)
	+ GPA: 3.5/4.0
	+ Relevant Coursework: Cloud Computing, Cybersecurity, Data Structures, Algorithms

**Skills:**
Primary Skills:
- Programming languages: Java, Python, C++
- Development frameworks: Spring, Django
- Databases: MySQL, PostgreSQL
Complementary Abilities:
- Communication skills
- Team management
- Time management

Technical Competencies (High Proficiency):
- Cloud platforms: AWS, Azure


In [11]:
# #Llama 3.3 with NVIDIA NIM
# client = ChatNVIDIA(
#   model="meta/llama-3.3-70b-instruct",
#   api_key="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", 
#   temperature=0.6
# )
# 
# 
# prompt = ChatPromptTemplate.from_template(resume_template)
# chain = prompt | model
# data = {
#     "name": fake.name(),
#     "company": df.iloc[i]["company"],
#     "job_role": df.iloc[i]["job_title"],
#     "jd": df.iloc[i]["job_summary"],
#     "skills":df.iloc[i]["job_skills"],
#     "fit_category": categories[2],
#     "category_requirements": category_requirements[categories[2]],
#     "resume_format" : resume_formats[random.choice([0, 1, 2])]
# }
# resume2 = chain.invoke(data)

In [12]:
# print(resume2)

In [13]:
#NVIDIA nemotron model listed as a model for synthetic data generation usecase
model = ChatNVIDIA(
  model="nvidia/nemotron-4-340b-instruct",
  api_key="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", 
  temperature=1.0,
        seed = fake.random_number()
)

In [14]:
# Good Fit
prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "name": fake.name(),
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[2],
    "category_requirements": category_requirements[categories[2]],
    "resume_format" : resume_formats[random.choice([0, 1, 2])]
}
resume3 = chain.invoke(data).content

In [15]:
data

{'name': 'Anthony Solomon',
 'company': 'Outdefine',
 'job_role': 'Data Engineer',
 'jd': "About Outdefine\nOutdefine is a web3 talent community that connects top talent with leading-edge companies and enterprises globally. Companies choose to hire Outdefine Trusted Members because their skills and readiness have been proven.\nWhen you accept a job as a Trusted Member, you will keep all of your pay. Contrast this with traditional hiring networks and agencies that charge membership fees and take up to 50% of your earnings as their markup. Additionally, Trusted Members get access to premier jobs, networking, and a global community powered by tokens. You can earn Outdefine tokens by working, contributing to the community, and referring friends.\nMore than 400 jobs are currently listed on Outdefine, with more being added regularly. Join over 40,000+ professionals from 115 countries who are building and developing their careers with Outdefine.\nIn order to apply for this position, first com

In [16]:
print(resume3)

**Anthony Solomon**
*Data Engineer*

*+1 (123) 456-7890* *|* *anthony.solomon@email.com* *|* *linkedin.com/in/anthony-solomon*

**Professional Summary**

Highly skilled Data Engineer with 10 years of experience in designing and building scalable data pipelines using Big Data stack, AWS, and Databricks. Expertise in Spark, Scala, and AWS S3, with a proven track record of delivering successful projects and driving business value.

**Skills**

*Primary Skills:* Spark, Scala, Databricks, AWS, AWS S3, Big Data, SQL, Cloud terminologies, AWS Glue ETL, Java, Python, RDD, Dataframes, Datasets, Streaming, Git Hub, Jenkins, Artifactory, CI/CD, Agile scrum, Talend Big Data, Unit testing, UAT, Prod Deployment, AWS Cloud, AWS Glue ETL, Databricks Delta, Redshift

*Technical Competencies:* Spark (Expert), Scala (Expert), Databricks (Expert), AWS (Expert), AWS S3 (Expert), Big Data (Advanced), SQL (Advanced), Cloud terminologies (Advanced), AWS Glue ETL (Advanced), Java (Intermediate), Python (Interm

We can clearly see that the NVIDIA's nemotron model generated the best content. Lets check it for other category fit.

In [17]:
#Complete Mismatch
data = {
    "name": fake.name(),
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[0],
    "category_requirements": category_requirements[categories[0]],
    "resume_format" : resume_formats[random.choice([0, 1, 2])]
}
r1 = chain.invoke(data).content
print(r1)

# Craig Fuller

*Email*: craig.fuller@email.com *Phone*: (123) 456-7890 *LinkedIn*: linkedin.com/in/craig-fuller

### Professional Summary

Highly skilled professional with 12+ years of experience in the automotive industry. Expertise in mechanical engineering, team leadership, and project management. Proven track record in optimizing manufacturing processes and reducing costs.

### Skills

*Mechanical Engineering, Automotive Design, CAD Software, Manufacturing Processes, Project Management, Team Leadership, Cost Reduction, Process Optimization, Lean Manufacturing, Quality Control, Welding Techniques, Supply Chain Management*

### Education

*University of Michigan*, Ann Arbor, MI

*Bachelor of Science in Mechanical Engineering*, 2008

*GPA*: 3.7/4.0

### Work Experience

*Senior Mechanical Engineer, AutoTech Solutions* (Jun 2018 - Present)

- Led a team of 10 engineers in developing and implementing new automotive designs, resulting in a 15% increase in fuel efficiency
- Managed a $2M

In [18]:
#Underwhelming
data = {
    "name": fake.name(),
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[1],
    "category_requirements": category_requirements[categories[1]],
    "resume_format" : resume_formats[random.choice([0, 1, 2])]
}
r2 = chain.invoke(data).content
print(r2)

# Carlos Mitchell

*Plano, TX* | *carlosmitchell@email.com* | *+1 (123) 456-7890* | *linkedin.com/in/carlosmitchell*

**PROFESSIONAL SUMMARY**

Data Engineer with 6+ years of experience in related industries, demonstrating a strong interest in transitioning to Big Data roles. Proficient in some required technologies, with a focus on continuous learning and development.

**SKILLS**

*Primary Skills*: Data Engineering, SQL, AWS, Java, Agile Scrum

*Technical Competencies*: Spark (proficient), Scala (basic), Databricks (basic), AWS S3, Git Hub, Jenkins, Artifactory, CI/CD, Talend Big Data, Unit testing, UAT, Prod Deployment

*Complementary Abilities*: Problem-solving, Communication, Teamwork, Adaptability

**WORK EXPERIENCE**

*Data Engineer, TechCo*

* *Plano, TX*
* *March 2019 - Present*

- Developed data pipelines using Java and AWS, improving data processing efficiency by 25%.
- Collaborated with cross-functional teams to implement CI/CD processes, reducing deployment time by 30%.
- M

In [19]:
data = {
    "name": fake.name(),
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[3],
    "category_requirements": category_requirements[categories[3]],
    "resume_format" : resume_formats[random.choice([0, 1, 2])]
}
r3 = chain.invoke(data).content
print(r3)

# Allison Morton

*Email*: allison.morton@email.com *Phone*: (123) 456-7890 *LinkedIn*: linkedin.com/in/allison-morton

**Professional Summary**

Highly skilled and experienced Data Engineer with 12+ years of expertise in designing and implementing scalable data solutions using Spark, Scala, Databricks, and AWS. Demonstrated leadership in managing and mentoring teams, driving strategic initiatives, and delivering high-impact results.

**Skills**

*Primary Skills*: Spark, Scala, Databricks, AWS, AWS S3, Big Data, SQL, Cloud terminologies, AWS Glue ETL, Java, Python, RDD, Dataframes, Datasets, Streaming, Git Hub, Jenkins, Artifactory, CI/CD, Agile scrum, Talend Big Data, Unit testing, UAT, Prod Deployment, AWS Cloud, AWS Glue ETL, Databricks Delta, Redshift

*Technical Competencies*: Expert in Hadoop ecosystem, NoSQL databases, data modeling, data warehousing, and ETL processes. Proficient in containerization technologies (Docker, Kubernetes) and cloud platforms (GCP, Azure).

*Complemen

NVIDIA nemotron seems to be the best model for data generation.

In [20]:
help(chain.invoke)

Help on method invoke in module langchain_core.runnables.base:

invoke(input: 'Input', config: 'Optional[RunnableConfig]' = None, **kwargs: 'Any') -> 'Output' method of langchain_core.runnables.base.RunnableSequence instance
    Transform a single input into an output. Override to implement.

    Args:
        input: The input to the Runnable.
        config: A config to use when invoking the Runnable.
           The config supports standard keys like 'tags', 'metadata' for tracing
           purposes, 'max_concurrency' for controlling how much work to do
           in parallel, and other keys. Please refer to the RunnableConfig
           for more details.

    Returns:
        The output of the Runnable.


In [21]:
help(chain)

Help on RunnableSequence in module langchain_core.runnables.base object:

class RunnableSequence(RunnableSerializable)
 |  RunnableSequence(*steps: 'RunnableLike', name: 'Optional[str]' = None, first: 'Optional[Runnable[Any, Any]]' = None, middle: 'Optional[list[Runnable[Any, Any]]]' = None, last: 'Optional[Runnable[Any, Any]]' = None) -> None
 |
 |  Sequence of Runnables, where the output of each is the input of the next.
 |
 |  **RunnableSequence** is the most important composition operator in LangChain
 |  as it is used in virtually every chain.
 |
 |  A RunnableSequence can be instantiated directly or more commonly by using the `|`
 |  operator where either the left or right operands (or both) must be a Runnable.
 |
 |  Any RunnableSequence automatically supports sync, async, batch.
 |
 |  The default implementations of `batch` and `abatch` utilize threadpools and
 |  asyncio gather and will be faster than naive invocation of invoke or ainvoke
 |  for IO bound Runnables.
 |
 |  Bat